**классификатор текстов**

SnowballStemmer+TfidfVectorizer

_Евгений Борисов <esborisov@sevsu.ru>_

## тексты

In [1]:
import pandas as pd
pd.options.display.precision = 2 
pd.options.display.max_colwidth = 200 

from tqdm.notebook import tqdm
tqdm.pandas()

In [2]:
# загружаем тексты
data = pd.read_pickle('../data/news.pkl.gz')
print('записей:',len(data))
data.sample(2)

записей: 3196


,text,tag
2216,"КИШИНЕВ, 13 дек — Sputnik. Силовики во вторник утром проверяют поезд сообщением Москва — Киев на Киевском вокзале в центре Москвы после анонимного сообщения о взрывном устройстве, сообщает РИА Нов...",incident
1479,Правящая партия и социал-демократы заявили о победе на выборах в Македонии\n\nМосква. 12 декабря. INTERFAX.RU - Правящая партия Внутренняя Македонская революционная организация - Демократическая п...,politics


## токенайзер со стемингом и очисткой

In [3]:
# простой токенайзер

import re
from razdel import tokenize
from nltk.corpus import stopwords as nltk_stopwords
stopwords = set(nltk_stopwords.words('russian'))

def tokenizer(text,stopwords=stopwords):
    return [
            t.text for t in tokenize( # разбиваем текст на слова
                re.sub(r'</?[a-z]+>',' ',text), # удаляем xml tag 
            ) 
            if not (
               False
               or (t.text in stopwords) # выкидываем предлоги, союзы и т.п.    
               or re.match(r'^[^a-zA-ZЁёА-я]+$', t.text) # выкидываем токены не содержащие букв
               or re.match(r'^(\w)\1+$', t.text)  # выкидываем токены из одного повторяющегося символа
               or re.match(r'^[^a-zA-ZЁёА-я].*$', t.text)  # выкидываем токены начинающиеся не с буквы
            )
        ] 

In [4]:
# токенайзер cо стеммером
#
# from nltk.tokenize import word_tokenize as nltk_tokenize_word
# from nltk.stem.snowball import SnowballStemmer
# from nltk.corpus import stopwords as nltk_stopwords
# import re

# stemmer = SnowballStemmer('russian')
# stopwords = set(nltk_stopwords.words('russian'))

# def tokenizer(text,stemmer=stemmer,stopwords=stopwords):
#     return [
#             stemmer.stem(t) # выполняем стеминг
#             for t in nltk_tokenize_word( # разбиваем текст на слова
#                 re.sub(r'</?[a-z]+>',' ',text), # удаляем xml tag 
#                 language='russian'
#             ) 
#             if not (
#                (len(t)<3) # выкидываем очень короткие слова
#                or re.match(r'^[^a-zA-ZЁёА-я]+$', t) # выкидываем токены не содержащие букв
#                or re.match(r'^(\w)\1+$', t)  # выкидываем токены из одного повторяющегося символа
#                or re.match(r'^[^a-zA-ZЁёА-я].*$', t)  # выкидываем токены начинающиеся не с буквы
#                or (t in stopwords) # выкидываем предлоги, союзы и т.п.    
#             )
#         ] 
    
# # data['text'].progress_apply(tokenizer)

In [5]:
# # токенайзер с лемматизацией

# from natasha import Doc
# from natasha import Segmenter
# from natasha import MorphVocab
# from natasha import NewsEmbedding
# from natasha import NewsMorphTagger

# from nltk.corpus import stopwords as nltk_stopwords
# stopwords = set(nltk_stopwords.words('russian'))

# seg = Segmenter() # базовый токенизатор
# # морфологический анализ
# tagger = NewsMorphTagger( NewsEmbedding() )
# lvoc = MorphVocab() # лемматизатор

# def tokenizer(text,seg=seg, tagger=tagger, lvoc=lvoc, stopwords=stopwords):
#     doc = Doc(text)
#     doc.segment(seg)
#     doc.tag_morph(tagger)
#     for t in doc.tokens: t.lemmatize(lvoc)
        
#     return [
#         t.lemma for t in doc.tokens
#         if not (
#              False
#             or (t.lemma in stopwords) # выкидываем предлоги, союзы и т.п.  
#             or re.match(r'^[^a-zA-ZЁёА-я]+$', t.lemma) # выкидываем токены не содержащие букв
#             or re.match(r'^(\w)\1+$', t.lemma)  # выкидываем токены из одного повторяющегося символа
#             or re.match(r'^[^a-zA-ZЁёА-я].*$', t.lemma)  # выкидываем токены начинающиеся не с буквы
#         )
#     ]

## выполняем частотный анализ

In [6]:
%%time

from sklearn.feature_extraction.text import TfidfVectorizer

# использования токенайзера вместе с векторайзером
tf_model = TfidfVectorizer(
        min_df=.001, # выкидываем очень редкие слова
        max_df=.10, # выкидываем очень частые слова
        use_idf=False, # не используем обратную частоту
        norm='l2', # нормируем TF
        tokenizer=tokenizer, # ф-ция токенайзер
        token_pattern=None, # отключаем дефолтный токенайзер
    )

data_tf = tf_model.fit_transform( data['text'] )

CPU times: user 5.49 s, sys: 12 ms, total: 5.5 s
Wall time: 5.51 s


In [7]:
data_tf.shape

(3196, 19991)

In [8]:
vcb1 = sorted(tf_model.vocabulary_)
print(len(vcb1))
pd.Series(vcb1).sample(30)

19991


18995                  хвост
931                   апреле
5243              зарубежной
7536                лозунгом
13789             производят
3042                    габи
13298            приземлился
5539                    иван
456                    trump
11326                 пикапа
5455                   знаки
19764    энергоэффективности
17337                    сэм
16962                 стоишь
18048                 уволен
18515                 утраты
16511          состоятельных
2674                   входе
3537                  грязью
4955                  заказа
10006             оказавшись
4236                   домах
18542              участвуют
5382                  звучат
16498              состоится
12419                 портит
1888                 версиям
7975                   места
539              абсолютного
4221                  долина
dtype: object

## формируем датасеты

In [9]:
labels = { t:i for i,t in enumerate(sorted(set(data['tag']))) }
labels

{'auto': 0,
 'culture': 1,
 'economics': 2,
 'health': 3,
 'incident': 4,
 'politics': 5,
 'realty': 6,
 'reclama': 7,
 'science': 8,
 'social': 9,
 'sport': 10,
 'tech': 11,
 'woman': 12}

In [10]:
y = data['tag'].map(labels).values
y

array([5, 1, 1, ..., 8, 5, 9])

In [11]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split( data_tf, y, test_size=0.3, random_state=326 )
X_train.shape, y_train.shape, X_test.shape, y_test.shape

((2237, 19991), (2237,), (959, 19991), (959,))

## обучаем классификатор

In [12]:
from sklearn.linear_model import SGDClassifier

clf = SGDClassifier(loss='hinge',max_iter=1000, tol=0.19).fit(X_train,y_train)

## тестируем

In [13]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score

In [14]:
# доля правильных ответов на учебном наборе
o = clf.predict(X_train)
accuracy_score(y_train,o)

0.9991059454626732

In [15]:
# доля правильных ответов на тестовом наборе
o = clf.predict(X_test)
accuracy_score(y_test,o)

0.8665276329509907

In [16]:
r = classification_report(
        y_test, o,
        output_dict=True,
        target_names=sorted(labels.keys())
    )

display(
    pd.DataFrame( { 
            k:v for k,v in r.items() 
            if not ( k in ['accuracy','macro avg','weighted avg'] ) 
        }).T.drop(columns=['f1-score']).rename(columns={'support':'количество'})
        .convert_dtypes()
        .style
           .background_gradient(cmap='RdYlGn', subset=['precision','recall'],)
           .bar(subset=['количество'],color=['#f00','#00ccff'])
)

,precision,recall,количество
auto,0.962025,0.873563,87
culture,0.762295,0.920792,101
economics,0.810127,0.842105,76
health,0.791667,0.612903,31
incident,0.897638,0.926829,123
politics,0.888298,0.943503,177
realty,0.882353,0.714286,21
reclama,1.000000,0.705882,17
science,0.894737,1.000000,68
social,0.538462,0.318182,44


| точность (precision) | полнота (recall) |
| :---: | :---: |
| $\frac{TP}{TP + FP}$ |  $\frac{TP}{TP + FN}$ |
| найдено позитивных / всего найдено |  найдено позитивных / всего позитивных |

In [17]:
pd.DataFrame( 
        confusion_matrix(y_test,o), 
        columns=sorted(labels.keys()), 
        index=sorted(labels.keys()),
    ).style.background_gradient(cmap='Blues')

,auto,culture,economics,health,incident,politics,realty,reclama,science,social,sport,tech,woman
auto,76,1,0,0,7,1,0,0,0,1,0,1,0
culture,0,93,0,1,0,4,0,0,0,1,0,2,0
economics,0,1,64,0,0,6,0,0,0,3,0,1,1
health,0,7,0,19,1,0,0,0,2,0,0,2,0
incident,1,0,1,2,114,1,0,0,0,1,0,3,0
politics,0,3,2,0,0,167,0,0,0,1,1,3,0
realty,0,1,2,0,0,1,15,0,0,1,0,1,0
reclama,0,1,2,0,0,0,1,12,0,1,0,0,0
science,0,0,0,0,0,0,0,0,68,0,0,0,0
social,1,10,5,1,4,3,1,0,1,14,0,4,0


---

In [18]:
data['predict'] = pd.Series( clf.predict(data_tf) ).map( { labels[k]:k for k in labels } )

In [19]:
data

,text,tag,predict
0,"В Саудовской Аравии сняли первый антитеррористический мультфильм -\nтрехминутную ленту ""Внимание!"". ""Внимание!"" отражает точку зрения мирового\nсообщества на войну, развязанную терроризмом, и поэт...",politics,culture
1,"Вчера вечером в Японии состоялась премьера голливудского фильма о гейшах,\nвызвавшая негодование в связи с тем, что эти девушки представлены\nпроститутками, а играющие их актрисы - китаянки. Мало ...",culture,culture
2,"Российский кинорежиссер и генеральный директор киноконцерна ""Мосфильм""\nКарен Шахназаров награжден ""Золотой пирамидой"" на XXIX Каирском кинофестивале\nза выдающийся вклад в мировое киноискусство. ...",culture,culture
3,30 ноября выдающейся российской балерине Майе Плисецкой будет вручена\nмедаль имени княгини Барборы Радвилайте. Церемония награждения состоится\nв Вильнюсе в Литовском национальном театре оперы и ...,culture,culture
4,"Гарольд Пинтер не приедет за Нобелевской премией из-за болезниАнглийский\nдраматург Гарольд Пинтер, получивший Нобелевскую премию по литературе в\n2005 году, отправит в Стокгольм видеозапись своей...",culture,culture
...,...,...,...
3191,Православие.Ру В сентябре 2010 года Святейший Патриарх Кирилл посетит Камчатку\nВопросы подготовки к поездке Предстоятеля Русской Православной Церкви на\nКамчатку обсуждались на встрече Святейшего...,social,social
3192,Интерфакс Религия (interfax-religion.ru) В Минрегионразвития призывают\nроссиян не бояться своего духовного наследия подобно Европе В министерстве\nрегионального развития РФ считают крайне важным ...,social,culture
3193,"Окно возможностей В Эвенкинском муниципальном районе приступили к\nисследованию традиционного уклада жизни. В Эвенкию прибыла экспедиция под\nруководством профессора, заведующего кафедрой менеджме...",science,science
3194,"ИТАР-ТАСС. Новости из властных структур. Совет Федерации предлагает определить\nособенности традиционной охоты коренных народов Севера, Сибири и Дальнего\nВостока Совет Федерации внес на рассмотр...",politics,politics


In [20]:
# Введение в анализ текстовой информации с помощью Python и методов машинного обучения
# https://habr.com/ru/post/205360/